In [1]:
import pandas as pd 
import operations as op
JCS_FILEPATH = "./../data/jcs_2020.csv"
SCIMAGO_FILEPATH = "./../data/scimagojr 2020.csv"

JCS_DF = pd.read_csv(JCS_FILEPATH, sep=";", low_memory=False).convert_dtypes()
SCIMAGO_DF = pd.read_csv(SCIMAGO_FILEPATH, sep=";", low_memory=False).convert_dtypes()


In [4]:
scimago_df = (SCIMAGO_DF
              .pipe(op.map, ['Title', 'Total Cites (3years)'])
              .pipe(op.rename_cols, columns = {
                    'Total Cites (3years)': 'Total Cites Scimago'
              })
              .pipe(op.apply_to_every_row, 'Title', lambda r: r["Title"].upper().strip())
              .pipe(op.convert_type, 'Total Cites Scimago', 'int64', 0))

jcs_df = (JCS_DF
          .pipe(op.map, ["Full Journal Title", "Total Cites"])
          .pipe(op.rename_cols, columns = {
                "Full Journal Title": "Title",
                "Total Cites": "Total Cites JCS"
          })
          .pipe(op.apply_to_every_row, 'Title', lambda r: r["Title"].upper().strip())
          .pipe(op.convert_type, "Total Cites JCS", 'str', '')
          .pipe(op.apply_to_every_row, "Total Cites JCS", lambda r: r["Total Cites JCS"].replace(',', ''))
          .pipe(op.convert_type, "Total Cites JCS", 'int64', 0))
          

journals_df = (scimago_df
               .pipe(op.join, df_right = jcs_df, left_on =["Title"], right_on =["Title"], how = 'outer')
               .pipe(op.apply_to_every_row, 'Cites', lambda r: 
                  r['Total Cites Scimago'] if r['Total Cites Scimago'] > r['Total Cites JCS'] else r['Total Cites JCS'])
               .pipe(op.map, ['Title', 'Cites'])
               .pipe(op.distinct))
journals_df

,Title,Cites
0,CA-A CANCER JOURNAL FOR CLINICIANS,55868.0
1,MMWR RECOMMENDATIONS AND REPORTS,3288.0
2,NATURE REVIEWS MOLECULAR CELL BIOLOGY,58477.0
3,QUARTERLY JOURNAL OF ECONOMICS,38427.0
4,NATURE REVIEWS MATERIALS,19887.0
...,...,...
36102,MICROBIAL PHYSIOLOGY,0.0
36103,OCEAN AND COASTAL RESEARCH,2.0
36104,ORNITHOLOGY RESEARCH,2.0
36105,STUDIES IN PSYCHOLOGY,9.0
